### Theano example implementation for data classification

This example should illustrate the functionality of Theano on the TensorFlow examples.

In [ ]:
# Importing all the things
%matplotlib notebook

import theano
import numpy as np
import theano.tensor as T
import matplotlib.pyplot as plt

from numpy import linalg as la

In [ ]:
from playground import DataGenerator

datasets = DataGenerator()

fig = plt.figure('Available Datasets')
for i, name in enumerate(['circle', 'xor', 'gauss', 'spiral']):
    data, labels = getattr(datasets, name)(200)
    axis = plt.subplot(221+i)
    axis.set_title(name)
    axis.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

In [ ]:
# debug settings (not that it would help a lot...)
theano.config.exception_verbosity='high'

The first implementation here shows a simple perceptron implementation in Theano:

In [ ]:
## LINEAR REGRESSION

from numpy import random as rnd

N = 400 # number of samples
feats = 20 # dimensionality of features
D = (rnd.randn(N, feats), rnd.randint(size=N, low=0, high=2)) # rand data
training_steps = 10000

x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rnd.randn(20), name="w")
b = theano.shared(0., name="b")

print("-------- Perceptron Model --------")
print(w.get_value(), b.get_value())

p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # probability that target = 1
prediction = p_1 > 0.5 # the prediction threshold
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # cross-entropy loss func
cost = xent.mean() + 0.01 * (w**2).sum() # the cost to minimize
gw, gb = T.grad(cost, [w, b])

train = theano.function(inputs = [x, y], outputs = [prediction, xent],
                        updates = {w : w-0.1*gw, b : b-0.1*gb})

predict = theano.function(inputs = [x], outputs = prediction)


for i in range(training_steps):
    pred, err = train(D[0], D[1])
    
print("-------- Training result -------")
print(w.get_value(), b.get_value())
print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))

Now let's extend it to the MLP with the data given above:

In [ ]:
## MULTILAYER-PERCEPTRON

# Choose your problem!
# datasets.circle(), datasets.xor(), datasets.gauss(), datasets.spiral()
n = 1000
data, labels = datasets.xor(n)
D = (data, labels.flatten())

x = T.matrix("x")
y = T.vector("y")
w_1 = theano.shared(rnd.randn(2,10), name="w1")
b_1 = theano.shared(np.zeros((10,)), name="b1")
w_2 = theano.shared(rnd.randn(10), name="w2")
b_2 = theano.shared(0., name="b2")

print("-------- Multilayer Model --------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

# expression graph for forward propagation
out = T.nnet.sigmoid(-T.dot(T.nnet.sigmoid(-T.dot(x,w_1)-b_1), w_2)-b_2)

prediction = out > 0.5

# cross entropy as loss function
cross_ent = -y * T.log(out) - (1-y) * T.log(1-out)
# cost function
cost = cross_ent.mean() + 0.01 * (w_2**2).sum()
gw_1, gb_1, gw_2, gb_2 = T.grad(cost, [w_1, b_1, w_2, b_2])

# compile the model
train = theano.function(inputs = [x, y], outputs = [prediction, cross_ent],
                        updates = [(w_1, w_1-0.1*gw_1), (b_1, b_1-0.1*gb_1),
                                   (w_2, w_2-0.1*gw_2), (b_2, b_2-0.1*gb_2)])

predict = theano.function([x], prediction)

epochs = 10000
batch_size = 100
for i in range(epochs):
    # get batches of training data
    b_indx = rnd.choice(n, batch_size)
    b_data = D[0][b_indx]
    b_target = D[1][b_indx]
    pred, err = train(D[0], D[1])
    
    if (i+1)%100 == 0:
        perf = 1 - sum(abs(predict(D[0]) - D[1])) / n
        print("Training... {:3.2%} done with accuracy {:2.2f}".format((i+1)/epochs, perf))
    

print("-------- Training result -------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))